In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
plt.rcParams['font.family'] = 'Gulim'

RuntimeError: module compiled against API version 0xe but this version of numpy is 0xd

In [2]:
from workalendar.asia import SouthKorea
import pendulum

In [23]:
train = pd.read_csv('../data/train.csv')
test = pd.read_csv('../data/test.csv')

## 전처리
- 일자에서 월과 일을 분리
- 요일을 레이블 인코딩화(EDA로 요일의 중요도 순 파악)
- 월 별, 일 별 중식 석식 수요 차이 파악

In [24]:
train['월'] = str(pd.DatetimeIndex(train['일자']).month)
test['월'] = str(pd.DatetimeIndex(test['일자']).month)
train['주'] = str(pd.DatetimeIndex(train['일자']).week)
test['주'] = str(pd.DatetimeIndex(test['일자']).week)
train['일'] = pd.DatetimeIndex(train['일자']).day
test['일'] = pd.DatetimeIndex(test['일자']).day

train['출근'] = train['본사정원수']-(train['본사휴가자수']+train['본사출장자수']+train['현본사소속재택근무자수'])
train['휴가비율'] = train['본사휴가자수']/train['본사정원수']
train['출장비율'] = train['본사출장자수']/train['본사정원수']
train['야근비율'] = train['본사시간외근무명령서승인건수']/train['출근']
train['재택비율'] = train['현본사소속재택근무자수']/train['본사정원수']

test['출근'] = test['본사정원수']-(test['본사휴가자수']+test['본사출장자수']+test['현본사소속재택근무자수'])
test['휴가비율'] = test['본사휴가자수']/test['본사정원수']
test['출장비율'] = test['본사출장자수']/test['본사정원수']
test['야근비율'] = test['본사시간외근무명령서승인건수']/test['출근']
test['재택비율'] = test['현본사소속재택근무자수']/test['본사정원수']

train['식사가능자수'] = train['본사정원수'] - train['본사휴가자수'] - train['현본사소속재택근무자수']
test['식사가능자수'] = test['본사정원수'] - test['본사휴가자수'] - test['현본사소속재택근무자수']

# train['중식참여율'] = train['중식계'] / train['식사가능자수']

<ipython-input-24-3deeb7b13432>:3: FutureWarning: weekofyear and week have been deprecated, please use DatetimeIndex.isocalendar().week instead, which returns a Series.  To exactly reproduce the behavior of week and weekofyear and return an Index, you may call pd.Int64Index(idx.isocalendar().week)
  train['주'] = str(pd.DatetimeIndex(train['일자']).week)
<ipython-input-24-3deeb7b13432>:4: FutureWarning: weekofyear and week have been deprecated, please use DatetimeIndex.isocalendar().week instead, which returns a Series.  To exactly reproduce the behavior of week and weekofyear and return an Index, you may call pd.Int64Index(idx.isocalendar().week)
  test['주'] = str(pd.DatetimeIndex(test['일자']).week)


In [25]:
def is_holiday(date):
    holidays = list(map(str, pd.Series(np.array(SouthKorea().holidays(int(date[:4])))[:, 0])))
    
    yesterday = str(np.datetime64(date) - 1)
    tomorrow = str(np.datetime64(date) + 1)

    if tomorrow in holidays and yesterday in holidays:
        return "3"
    if tomorrow in holidays:
        return "2"
    elif yesterday in holidays:
        return "1"
    else : 
        return "0"

def week_of_month(x):
    dt = pendulum.parse(x)
    
    wom = dt.week_of_month
    if wom < 0:
        wom += 52
    return wom
    

df = pd.concat([train[['본사정원수', '일자']], test[['본사정원수', '일자']]])
df['년월'] = df['일자'].apply(lambda x : x[:7])
df = df[['년월', '본사정원수']].groupby(by=['년월'], as_index=False).mean()

def member_change(date):
    this_month = date[:7]
    last_month = str(np.datetime64(this_month) - 1)
    
    this_month_member = int(df[df['년월'] == this_month]['본사정원수'])
    last_month_member = int(df[df['년월'] == last_month]['본사정원수'])
    
    
    return  this_month_member - last_month_member

train['공휴일전후'] = train['일자'].apply(is_holiday)
test['공휴일전후'] = test['일자'].apply(is_holiday)

train['몇주차'] = train['일자'].apply(week_of_month)
test['몇주차'] = test['일자'].apply(week_of_month)

train = train[train['일자'] > '2016-03']
train['인원변화'] = train['일자'].apply(member_change)
test['인원변화'] = test['일자'].apply(member_change)

train['day2']="0"
train.loc[train['일']>9, 'day2'] = "1"
train.loc[train['일']>19, 'day2'] = "2"

test['day2']="0"
test.loc[test['일']>9, 'day2'] = "1"
test.loc[test['일']>19, 'day2'] = "2"

## 공휴일 변수 생성

In [26]:
train.columns

Index(['일자', '요일', '본사정원수', '본사휴가자수', '본사출장자수', '본사시간외근무명령서승인건수',
       '현본사소속재택근무자수', '조식메뉴', '중식메뉴', '석식메뉴', '중식계', '석식계', '월', '주', '일',
       '출근', '휴가비율', '출장비율', '야근비율', '재택비율', '식사가능자수', '공휴일전후', '몇주차', '인원변화',
       'day2'],
      dtype='object')

In [27]:
# 메뉴 변수 없이 사용할떄 해당 코드 사용['공휴일전후', '몇주차', '인원변화']

lunch_train = train[['공휴일전후', '몇주차', '인원변화', '요일','월','일','주','출근', '휴가비율', '출장비율', '야근비율', '재택비율','본사출장자수','본사휴가자수','식사가능자수','본사시간외근무명령서승인건수']]
lunch_test = test[['공휴일전후', '몇주차', '인원변화', '요일','월','일','주','출근', '휴가비율', '출장비율', '야근비율', '재택비율','본사출장자수','본사휴가자수','식사가능자수','본사시간외근무명령서승인건수']]

dinner_train= train[['공휴일전후', '몇주차', '인원변화', '요일','월','일','주','출근', '휴가비율', '출장비율', '야근비율', '재택비율','본사출장자수','본사휴가자수','식사가능자수','본사시간외근무명령서승인건수']]
dinner_test = test[['공휴일전후', '몇주차', '인원변화', '요일','월','일','주','출근', '휴가비율', '출장비율', '야근비율', '재택비율','본사출장자수','본사휴가자수','식사가능자수','본사시간외근무명령서승인건수']]

In [28]:
lunch_train.columns

Index(['공휴일전후', '몇주차', '인원변화', '요일', '월', '일', '주', '출근', '휴가비율', '출장비율',
       '야근비율', '재택비율', '본사출장자수', '본사휴가자수', '식사가능자수', '본사시간외근무명령서승인건수'],
      dtype='object')

In [29]:
print(lunch_train.shape)
print(lunch_test.shape)

(1187, 16)
(50, 16)


In [30]:
print(dinner_train.shape)
print(dinner_test.shape)

(1187, 16)
(50, 16)


In [31]:
cat_features = [f for f in lunch_train.columns if lunch_train[f].dtype == 'object']

def column_index(df, cat_features):
    cols = df.columns.values
    sidx = np.argsort(cols)
    return sidx[np.searchsorted(cols, cat_features, sorter=sidx)]

cat_features_idx = column_index(lunch_train, cat_features)    
print("Cat features are: %s" % [f for f in cat_features])
print(cat_features_idx)

Cat features are: ['공휴일전후', '요일', '월', '주']
[0 3 4 6]


In [32]:
y_lunch = train[['중식계']]
y_dinner = train[['석식계']]

#### 분포 확인 및 분포 조정

# 중식 예측모델

In [54]:
from automl_alex import LightGBMRegressor, CatBoostRegressor, AutoMLRegressor
from sklearn.metrics import mean_absolute_error as MAE
import sklearn

In [57]:
lunch_model = CatBoostRegressor(random_state=42)

lunch_model.opt(lunch_train,  y_lunch,
            verbose=3,  cold_start=120,folds=8,opt_lvl=3,early_stoping=120, auto_parameters=False,#metric=sklearn.metrics.roc_auc_score,
      timeout=1100,metric=sklearn.metrics.mean_absolute_error # optimization time in seconds,
      )

21:25:04 | regression optimize: minimize
--- Logging error in Loguru Handler #6 ---
Record was: {'elapsed': datetime.timedelta(seconds=3380, microseconds=636107), 'exception': None, 'extra': {}, 'file': (name='_base.py', path='C:\\anaconda3\\envs\\machinelearning\\lib\\site-packages\\automl_alex\\_base.py'), 'function': 'opt', 'level': (name='INFO', no=20, icon='ℹ️'), 'line': 270, 'message': 'regression optimize: minimize', 'module': '_base', 'name': 'automl_alex._base', 'process': (id=10636, name='MainProcess'), 'thread': (id=6924, name='MainThread'), 'time': datetime(2021, 7, 15, 21, 25, 4, 488926, tzinfo=datetime.timezone(datetime.timedelta(seconds=32400), '대한민국 표준시'))}
Traceback (most recent call last):
  File "C:\anaconda3\envs\machinelearning\lib\site-packages\loguru\_handler.py", line 177, in emit
    self._sink.write(str_record)
  File "C:\anaconda3\envs\machinelearning\lib\site-packages\loguru\_file_sink.py", line 174, in write
    self._terminate_file(is_rotating=True)
  File

KeyError: 'CatBoostRegressor'

In [58]:
predicts_Auto_lunch = lunch_model.predict(lunch_test)

Exception: No fit models

In [ ]:
lunch_model.get_model_opt_params()

In [53]:
dinner_model = CatBoostRegressor(random_state=42)

dinner_model.opt(lunch_train,  y_lunch,
                verbose=3,  cold_start=120,folds=8,opt_lvl=3,early_stoping=120, auto_parameters=False,#metric=sklearn.metrics.roc_auc_score,
              timeout=1100,metric=sklearn.metrics.mean_absolute_error # optimization time in seconds,
              )

21:24:13 | regression optimize: minimize
--- Logging error in Loguru Handler #2 ---
Record was: {'elapsed': datetime.timedelta(seconds=3329, microseconds=467113), 'exception': None, 'extra': {}, 'file': (name='_base.py', path='C:\\anaconda3\\envs\\machinelearning\\lib\\site-packages\\automl_alex\\_base.py'), 'function': 'opt', 'level': (name='INFO', no=20, icon='ℹ️'), 'line': 270, 'message': 'regression optimize: minimize', 'module': '_base', 'name': 'automl_alex._base', 'process': (id=10636, name='MainProcess'), 'thread': (id=6924, name='MainThread'), 'time': datetime(2021, 7, 15, 21, 24, 13, 319932, tzinfo=datetime.timezone(datetime.timedelta(seconds=32400), '대한민국 표준시'))}
Traceback (most recent call last):
  File "C:\anaconda3\envs\machinelearning\lib\site-packages\loguru\_handler.py", line 177, in emit
    self._sink.write(str_record)
  File "C:\anaconda3\envs\machinelearning\lib\site-packages\loguru\_file_sink.py", line 174, in write
    self._terminate_file(is_rotating=True)
  Fil

KeyError: 'CatBoostRegressor'

In [44]:
lunch_train.columns, lunch_test.columns, 

(Index(['공휴일전후', '몇주차', '인원변화', '요일', '월', '일', '주', '출근', '휴가비율', '출장비율',
        '야근비율', '재택비율', '본사출장자수', '본사휴가자수', '식사가능자수', '본사시간외근무명령서승인건수'],
       dtype='object'),
 Index(['공휴일전후', '몇주차', '인원변화', '요일', '월', '일', '주', '출근', '휴가비율', '출장비율',
        '야근비율', '재택비율', '본사출장자수', '본사휴가자수', '식사가능자수', '본사시간외근무명령서승인건수'],
       dtype='object'))

In [45]:
predicts_Auto_dinner = dinner_model.predict(dinner_test)

In [46]:
submission = pd.read_csv('../data/sample_submission.csv')

In [47]:
submission.iloc[:,1] = predicts_Auto_lunch
submission.iloc[:,2] = predicts_Auto_dinner
submission.head()

,일자,중식계,석식계
0,2021-01-27,1022.000418,386.208373
1,2021-01-28,942.895740,446.545413
2,2021-01-29,606.124002,226.351950
3,2021-02-01,1285.863560,506.678670
4,2021-02-02,1057.748842,453.229518


In [50]:
def compare_ans(DIR):
    answer = pd.read_csv(DIR)

    lunch_answer = np.array(answer.iloc[:,1])
    dinner_answer = np.array(answer.iloc[:,2])
    
    lunch_MAE = abs(predicts_Auto_lunch - lunch_answer).mean()
    dinner_MAE = abs(predicts_Auto_dinner - dinner_answer).mean()
    
    print("lunch_MAE : ", lunch_MAE)
    print("dinner_MAE : ", dinner_MAE)
    print("total_MAE : ", (lunch_MAE+dinner_MAE)/2)
    
    
compare_ans('../submission/20210715_lgbm_autoML.csv')
compare_ans('../submission/20210715_lgbm_autoML_ensenble.csv')

lunch_MAE :  25.385260160973232
dinner_MAE :  43.00003275954029
total_MAE :  34.19264646025676
lunch_MAE :  34.62573286811751
dinner_MAE :  49.91989531195473
total_MAE :  42.27281409003612


In [51]:
import math

math.cos(math.pi*68.44/180)*69.5

25.539537282882215

# 저장

In [52]:
import datetime
today = str(datetime.datetime.now().date()).replace("-","")
print("오늘 날짜 : " + today)

submission.to_csv(f'../submission/{today}_cat_autoML.csv', index =False)

오늘 날짜 : 20210715


In [69]:
answer = pd.read_csv('../submission/20210715_pycaret_(2).csv')

In [84]:
best_submission = pd.read_csv('../data/sample_submission.csv')
best_submission.iloc[:,1:] = submission.iloc[:,1:]*5/9 +  answer.iloc[:,1:]*4/9
best_submission.to_csv(f'../submission/{today}_lgbm_autoML_ensenble.csv', index =False)